In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os


local_model_path = "llama2-7b-chat"  # or full path if you prefer
current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
path = os.path.join(parent_path, local_model_path)

device = "mps" if torch.backends.mps.is_available() else "cpu"
print("Using device:", device)

# Load tokenizer from local dir
tokenizer = AutoTokenizer.from_pretrained(path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    path,
    dtype = torch.float16,
    local_files_only = True
    )

model.to(device)
print('model to divice')

/Users/koorosh/Documents/Coding/Foundation_Model/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: mps


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 25.65it/s]


model to divice


In [5]:
prompt = (
    "Tell what is optoacosutic."
)

input = tokenizer(prompt, return_tensors = 'pt').to(device)

with torch.no_grad():
        
    output_ids = model.generate(
        **input,
        max_new_tokens = 200,
        do_sample = True,
        temperature = 0.7,
        top_p = 0.9,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_time=20,
    )

output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)

Tell what is optoacosutic.
Optoacoustic refers to the use of light (opto) to generate sound (acoustic) for imaging or sensing applications. Optoacoustic techniques use light to excite molecules or particles in a sample, which then emit sound waves that can be detected and used to create images or measure properties of the sample. This approach offers several advantages over traditional imaging methods, including higher resolution, deeper penetration, and non-invasive imaging.

In optoacoustic imaging, light is directed at a sample, and the absorption or scattering of the light by the sample causes the sample to heat up or vibrate. These changes in temperature or pressure generate sound waves that can be detected using an ultrasound transducer. The resulting image shows the distribution of light absorption or scattering in the sample.

Optoacoustic techniques have been used in a variety of applications, including


In [ ]:
from datasets import Dataset
examples = [
    {
        "instruction": "Explain overfitting in machine learning.",
        "input": "",
        "output": "Overfitting happens when a model memorizes the training data instead of learning general patterns, so it performs well on training data but poorly on new, unseen data."
    },
    {
        "instruction": "Give me 3 tips to study more effectively.",
        "input": "",
        "output": "1) Study in focused blocks with breaks, 2) test yourself instead of only rereading, 3) sleep well to consolidate memory."
    },
    {
        "instruction": "Explain what a GPU is to a 10-year-old.",
        "input": "",
        "output": "A GPU is like a super helper in your computer that can do many small math problems at the same time, which makes games and AI run faster."
    },
]

raw_dataset = Dataset.from_list(examples)

def format_example(example):
    instruction = example['instruction']
    if example.get("input"):
        prompt = f"Instruction: {instruction}\nInput: {example['input']}\nResponse:"
    else:
        prompt = f"instruction: {instruction}\nResponse:"
    prompt = f"{prompt} {example['output']}"

    tokenized = tokenizer(
        prompt,
        truncation=True,
        max_length= 200,
    )
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

dataset_ids = raw_dataset.map(format_example)
print(dataset_ids[0])

In [24]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r = 8,
    lora_alpha= 16,
    lora_dropout= 0.05,
    target_modules= ['q_proj', 'k_proj'],
    task_type= 'CAUSAL_LM'
)
model_lora = get_peft_model(model, lora_config)
model_lora.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.0622
